In [1]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import math

import pymorphy2
import re
from collections import defaultdict

from tqdm import tqdm, tqdm_notebook
import bisect
from collections import namedtuple,defaultdict
import numpy as np


import lxml.html
import lxml.etree
import requests
import sqlite3

DocEntry = namedtuple('DocEntry',['doc_id','positions'])

In [2]:
conn = sqlite3.connect("news_base.db") 
cursor = conn.cursor()
"""создание тестовой выборки, чтобы проверить качетсво кластеризации
"""

d=[] #список документов для дальнейшей работы
doc_id = 0

stopWords = stopwords.words('russian')
stopWords.extend(['что', 'это', 'так', 'вот', 'быть','й', 'как', 'в', '—', 'к', 'на','свой', 'который', 'е', 'также','год'])

test_doc_to_clast = defaultdict(list)

categories = ['Культура','Наука и техника','Спорт','Мир', 'Финансы','Бывший СССР']
""" 0 - культура , 1 -наука, 3 - спорт"""
for n_clast,  c in enumerate(categories):
    sql = "SELECT article FROM News where category = ? limit 1000"
    cursor.execute(sql,[c] )
    
    for t in cursor.fetchall()[:100]:
        
        test_doc_to_clast[n_clast].append(doc_id)
        article = t[0]
        d.append(article)
        #print( n_clast, c, doc_id,article,"\n")
        doc_id+=1

x = np.array(d) 

In [3]:
""" инвертированный индекс 
    используется для построения матриц вероятностей 
    """


def parse_text(text):
    words = (word for word in re.split('\W+', text) if (len(word) >0 )&( analyzer.normal_forms(word)[0].lower() not in stopWords ))
    lexems = (analyzer.normal_forms(word)[0] for word in words)
    
    return list(lexems)


class InvertedIndex:
    
    def __init__(self):
        self.dict = defaultdict(list)
        self.texts = dict()
        
    def add_document(self, text, doc_id):
        self.texts[doc_id] = text
        words = parse_text(text)
        
        
        word_to_entry = defaultdict(lambda: [])
        
        for pos, word in enumerate(words):
            
            doc_entry = word_to_entry[word]
            doc_entry.append(pos)
            
        for word, positions in word_to_entry.items():
            postings = self.dict[word]
            entry = DocEntry(doc_id, positions)
            
            i = bisect.bisect_left(postings, entry)
        
            postings.insert(i, entry)
            
    def get_postings(self, word):
        return self.dict[word]
    

        
"""Подсчет векторного произведения"""        
def mult_single(x,y):
    #conditions x>0 y>o
    return x*y

def mult_vect(x,y):
    mult = np.vectorize(mult_single)
    return np.sum(mult(x,y))

"""Dkl"""

def dkl_single(x,y):
    if x>0 and y>0:
     
        return x*math.log2(x/y)
    elif x==0: return 0.0
    else: return math.inf

def dkl(x,y):
    sum_dkl = np.vectorize(dkl_single)
    
    return np.sum(sum_dkl(x,y))

def div_matr(D, n, m,py_x,py_t) :
    for i in range(n):
        for j in range(m):
            Div[i,j] = dkl(py_x[:,i],py_t.transpose()[:,j]) 
            
    return Div
    
"""Djs"""
def djs(x,y):
    return 0.5 * dkl(x, (x+y)/2) + 0.5* dkl(y, (x+y)/2)

def djs_vect(Djs,m,pt_x,pt_x_new ):
    for i in range(m):
        
        Djs[i]= djs(pt_x[i,:], pt_x_new[i,:])
    return Djs

"""P(t)"""
def f_pt(pt,m,px,pt_x):
    for i in range(m):
        pt[i] = mult_vect(px, pt_x[i,:])
    return pt

"""P(y|t)"""
def f_py_t(py_t, m,n,pt_x,pxy,pt):
    for i in range(m):
        for j in range(n):
            py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
            
    return py_t
"""Выводит список из  n элементов, соответсвующих  """
def main_words(n, r):
    l = []

    for i in range(n):
        l.append(r.argmax())
        r[r.argmax()] = 0
    return l


        
#show all numbers of docs from  cl,   
def show_text_docs(cl_num):
    show_main_words(cl_num)
    print("\n")
    docs = doc_clast[cl_num] # список документов кластера №cl_num
    for num in docs:
        print("№ документа: " + str(num) + " ТЕКСТ "+ str(x[num]))
    
    
    
def foo_z(pt_x):
    z = pt_x.sum(axis=0)
    return z


#w_d список документов со словом w
#d_cl список документов из кластера t 

#w_d,  d_cl
def f_N11( w_d,d_cl,n):
    k= 0
    
    for d in range(n):
        if (d in d_cl )& (d in w_d):
            k+=1
    return k
#not w_d, w_cl
def f_N01(w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d in w_cl )& (d not in w_d):
            k+=1
    return k
#not w_d, not w_cl
def f_N00( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d not in w_d):
            k+=1
    return k

#w_d,   not w_cl
def f_N10( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d in w_d):
            k+=1
    return k

def mutual_information(t):
    MI = []
    for word in (word_dict.items()):
        w  = word[1]
        
        w_d=[]
        for doc_id,pos in index.dict.get(w):
            w_d.append(doc_id)
        N = len(x)
        N11 = f_N11(w_d,doc_clast.get(t),len(x))
        N1 = len(w_d)
        N10 = f_N10(w_d,doc_clast.get(t),len(x))
        N0 = N - N1
        #print(N, N1, N0)
        N01 = f_N01(w_d,doc_clast.get(t),len(x))
        N00 = f_N00(w_d,doc_clast.get(t),len(x))
        if N10==0: N10=0.0001
        if N01==0: N01=0.0001
        if N11==0: N11=0.0001
        if N00==0: N00=0.0001
   
        #print(N11, N01, N10, N00)
        I = N11/N * math.log2(N*N11/(N1*N1)) + N01/N*math.log2(N * N01/(N0*N1)) +N10/N*math.log2(N * N10/(N0*N1))+N00/N*math.log2(N * N00/(N0*N0))
        MI.append(I)
    return MI
    
def main_words(n, r):
    l = []
    
    for i in range(n): 
        maxi = r.argmax()
        l.append((maxi,r[maxi]))
        r[maxi] = 0
    return l

In [35]:
#beta
beta = 2.11

#число кластеров
m = 6
#covergence parameter 

e = 0.01

In [36]:
"""INPUT"""

analyzer = pymorphy2.MorphAnalyzer()   
"""Создание инвертированного индекса"""   
index = InvertedIndex()

for i, line in enumerate(x):
        doc_id = i
        text = line
        index.add_document(text,doc_id)
        
#словарь слов: ключ - номер индекса , значение -словао
word_dict = {}
for i, word in enumerate(index.dict.keys()):
    word_dict[i] = word

"""общее число слов в коллекции"""
summa = 0
for i in x:
    #print(len(word_tokenize(i)))
    summa+=len(word_tokenize(i))

"""P(x,y)"""    
pxy = np.zeros((len(x), len(index.dict.keys())))
for i, word in enumerate(index.dict.keys()):
    a = index.dict.get(word)
    for doc_id , postings in a:
        pxy[doc_id,i] = len(postings)/summa
"""print("pxy")
print(pxy)"""     
        
"""INITIALIZaTION""" 

"""P(t|x)"""
pt_x = np.random.rand(m, len(x))

z = pt_x.sum(axis=0)
pt_x = pt_x/z
#print(pt_x)
    
"""задание вероятности P(x)"""    
px  = np.zeros((x.size))

for n, i in enumerate(x):
    b = len(word_tokenize(i))
    #print(i, b, n)
    px[n] = b/summa
#print("px")
#print(px)
    


"""P(t)"""
pt = np.zeros(m)
for i in range(m):
    pt[i] = mult_vect(px,pt_x[i,:])
    
"""P(y|t)"""
py_t = np.zeros((m,len(index.dict.keys())) )

for i in range(m):
    for j in range(len(index.dict.keys())):
        py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
        

"""P(y|x)"""
py_x = np.zeros((len(index.dict.keys()),len(x)))


word_index=[]
for i, word in enumerate(index.dict.keys()):
    a = index.dict.get(word)
    word_index.append((i,word))
    for doc_id , postings in a:
        py_x[i,doc_id] = len(postings)/len(index.texts[doc_id].split())
        

      
        """LOOP"""
#print("y_x")
#print(py_x) 
Div = np.zeros((len(x), m)) #init div   
Div = div_matr(Div, len(x),m,py_x,py_t)
#new P(t|x)
#z = f_z(pt,py_x,py_t,beta)

pt_x_new =pt * np.exp(-beta * Div)
pt_x_new = pt_x_new.transpose()
#print("pt_x_new")
#print(pt_x_new)
z = foo_z(pt_x_new)
#print("z")
#print(z)
pt_x_new = pt_x_new/z
#print("pt_x_new")
#print(pt_x_new)

#init Djs
Djs = np.zeros(m)
Djs = djs_vect(Djs,m,pt_x,pt_x_new )

k = 0
"""print("pt_x")
print(pt_x)
print("pt")
print(pt)
print("py_t")
print(py_t)
print("DJS")
print(Djs)"""

while np.sum(Djs)/len(Djs)>=e:
    print(k)
    #переопрделяем 
    pt_x = pt_x_new 
    """print("pt_x")
    print(pt_x)"""
    #P(t)    
    pt = f_pt(pt,m,px,pt_x)
    """print("pt")
    print(pt)"""
    
    #P(y|t)
    py_t = f_py_t(py_t, m,len(index.dict.keys()),pt_x,pxy,pt) 
    """print(py_t)
    print(py_t)"""
                  
    #P(t|x) new
    pt_x_new = pt * np.exp(-beta * div_matr(Div, len(x),m,py_x,py_t)) 
    """print('pt_new')
    print(pt_x_new)"""
    pt_x_new = pt_x_new.transpose()
    z = foo_z(pt_x_new)
    pt_x_new = pt_x_new/z
    """print("pt_x_new")
    print(pt_x_new)"""
       
    Djs = djs_vect(Djs,m,pt_x,pt_x_new )
    #print("DJS")
    #print(Djs)
    k+=1

# вектор с номерами кластеров, к которому документ предложит( вероятность больше всего)
vec = np.argmax(pt_x_new, axis=0)

#словарь: ключ - номер кластера, значение - номер документа
doc_clast = defaultdict(list)
for i,value in enumerate(vec):
        doc_clast[value].append(i)

        
#СЛОВАРЬ: ключ - номер кластера, значение - список и n самых важных слов

clasters_main_words = {}
n = 50
py_t_copy = py_t.copy()
for i, row in enumerate(py_t_copy):  
    
    clasters_main_words[i] = main_words(n, row) 
    #print(clasters_main_words[i])
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [70]:
t = 0
MI = mutual_information(t)

In [ ]:
MainWords=[]

for i,weight in main_words(1000, np.array(MI.copy())):
    MainWords.append((i,word_dict[i],weight))
MainWords 



In [76]:
len(x)
doc_clast[0]
x[208]

"['главный тренер мадридского «реала» зинедин зидан перед финальным матчем лиги чемпионов с туринским «ювентусом» решил мотивировать футболистов, показав им фильм. об этом сообщает marca.', 'французский специалист выбрал картину 2006 года режиссера зака снайдера «300 спартанцев». игрокам будут продемонстрированы самые яркие фрагменты фильма. тренер хочет, чтобы его футболисты прониклись духом спартанского царя леонида и его воинов. по его мнению, просмотр киноленты будет эффективнее мотивационных речей.', 'фильм основан на комиксах фрэнка миллера. речь в картине идет о 300 спартанцах, которые остановили многотысячную армию, ведомую персидским царем ксерксом.', 'финальная встреча лиги чемпионов между «реалом» и «ювентусом» пройдет в субботу, 3 июня. игра состоится в кардиффе на стадионе «миллениум». матч начнется в 21:45 по московскому времени.']"

In [67]:
#подсчет статистики
l2  = test_doc_to_clast[2] 
l1 = doc_clast[5]

result=list(set(l1) & set(l2))
print(len(result))

71


In [84]:
#словарь слово ключ : индекс слова, значение : вес
mi_dict={}
for i, mi in enumerate(MI):
    mi_dict[i] = mi
    
    
def sent_weight(l):  #возвращает вес предложения
    weight = 0
    for word in l:
        weight+=mi_dict[new_word_dict[word]]
    return weight

# переопределение словаря, теперь ключ: слово, значение :индекс
new_word_dict = {}
for i, word in enumerate(index.dict.keys()):
    new_word_dict[word] = i  
    

    

In [85]:
# считаем веса каждого предложения новости
data = x[208]
sentences = sent_tokenize(data)
sentence_weight=[]
for i,sent in enumerate(sentences):
    parsed_text = parse_text(sent)
    sentence_weight.append(sent_weight(parsed_text))

In [88]:
#добавить сортировку 
for i in sentence_weight:
    print(i)

10.782580727012224
1.44102062952326
6.900902070518959
2.6685637708610797
8.390562526684695
4.371469147926859
3.4736381993567758
10.87011029131903
4.079230897220543
3.213912740981539
1.6384854554490085


In [90]:
print(sentences[0])
print(sentences[2])
print(sentences[4])
print(sentences[7])

['главный тренер мадридского «реала» зинедин зидан перед финальным матчем лиги чемпионов с туринским «ювентусом» решил мотивировать футболистов, показав им фильм.
', 'французский специалист выбрал картину 2006 года режиссера зака снайдера «300 спартанцев».
тренер хочет, чтобы его футболисты прониклись духом спартанского царя леонида и его воинов.
речь в картине идет о 300 спартанцах, которые остановили многотысячную армию, ведомую персидским царем ксерксом.


In [68]:
#распределение документов по кластерам
pt

array([ 0.14949203,  0.2258139 ,  0.09211367,  0.21299509,  0.13915805,
        0.18042726])

In [85]:
pt_x[:,340]

array([  1.41317602e-04,   4.87777507e-01,   1.12109934e-06,
         1.64097961e-01,   3.01598758e-01,   4.63833358e-02])